In [1]:
import tensorflow as tf
import numpy as np
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 
            'h', 'i', 'j', 'k', 'l', 'm', 'n', 
            'o', 'p', 'q', 'r', 's', 't', 'u', 
            'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
# 학습에 사용될 단어
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
# 학습에 사용할 ㅅ ㅜ있는 형식으로 변환 
# 1. 입력값용으로, 단어의 처음 세글자의 알파벳 인덱스를 구한 배열을 만듬
# input=[num_dic[n] for n in seq[:-1]]
# 2. 출력값용으로, 마지막 글자의 알파벳 인덱스를 구함
# target = num_dic[seq[-1]]
# 3. 입력값을 원-핫 인코딩으로 변환
# input_batch.append(np.eye(dic_len)[input])
# 실측값은 p라면 15로 나오며(순번), sparse_softmax_cross_entropy_with_logits를 사용할것이기 때문에 그대로 사용 
# 위 함수는 실측값, 즉 labels 값에 원-핫 인코딩을 사용하지 않다도 자동으로 변환하여 계산해줌

def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

# 신경망 모델 구성
learning_rate = 0.01
n_hidden = 128
total_epoch =30

n_step = 3
n_input = n_class = dic_len

# sparse_softmax_cross_entropy_with_logits함수를 사용할때 실측값인 labels값은 인덱스의 숫자를 그대로 사용하고
# 예측모델의 출력값은 인덱스의 원-핫 인코딩을 사용
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))
# DropoutWrapper함수를 사용하여 RNN에도 과적합 방지를 위한 드롭아웃 기법을 쉽게 적용할 수 있음
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 만든 셀들을 MultiRNNCell함수를 사용하여 조합
# dynamic_rnn 함수를 사용하여 심층 순환 신경망, Deep RNN을 만듬
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype= tf.float32)

# MNIST 예측 모델과 같은 방식으로 최종 출력층을 만듬
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    print('Epoch:', '%04d' % (epoch +1), 'cost =', '{:.6f}'.format(loss))

print('최적화 완료')

prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict ={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


Epoch: 0001 cost = 2.549237
Epoch: 0002 cost = 1.867465
Epoch: 0003 cost = 1.276461
Epoch: 0004 cost = 0.814797
Epoch: 0005 cost = 0.638673
Epoch: 0006 cost = 0.486418
Epoch: 0007 cost = 0.560993
Epoch: 0008 cost = 0.313843
Epoch: 0009 cost = 0.349508
Epoch: 0010 cost = 0.246192
Epoch: 0011 cost = 0.177897
Epoch: 0012 cost = 0.183945
Epoch: 0013 cost = 0.282522
Epoch: 0014 cost = 0.153408
Epoch: 0015 cost = 0.061263
Epoch: 0016 cost = 0.110521
Epoch: 0017 cost = 0.112558
Epoch: 0018 cost = 0.033661
Epoch: 0019 cost = 0.303052
Epoch: 0020 cost = 0.017280
Epoch: 0021 cost = 0.040136
Epoch: 0022 cost = 0.085883
Epoch: 0023 cost = 0.019499
Epoch: 0024 cost = 0.065269
Epoch: 0025 cost = 0.029937
Epoch: 0026 cost = 0.043731
Epoch: 0027 cost = 0.006156
Epoch: 0028 cost = 0.002879
Epoch: 0029 cost = 0.005127
Epoch: 0030 cost = 0.005906
최적화 완료

=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool'